# Download To Google Drive 

Let's download files directly to your preferred Google Drive folder from Remote URLs.
<br>

*   Run below cells sequentially,
*   Follow display instructions.



### Mount your Google Drive

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

### Select download location


In [ ]:
import os
os.chdir("/content/drive/My Drive/downloads")
print('Download location selected.')



---



###Let's download

In [ ]:
import os
import posixpath
import requests
import time
import sys
try:
    from urlparse import urlsplit
    from urllib import unquote
except ImportError: # Python 3
    from urllib.parse import urlsplit, unquote

links = []
last_index = (len(links) - 1)
download_ok = True


# get filename
def url_to_filename(file_url):
    urlpath = urlsplit(file_url).path
    basename = posixpath.basename(unquote(urlpath))
    if (os.path.basename(basename) != basename or
        unquote(posixpath.basename(urlpath)) != basename):
        raise ValueError
    return basename


# download function
def get_downloads(file_download_url):
    try:
        current_filename = url_to_filename(file_download_url)
        start = time.perf_counter()
        
        with open(current_filename, "wb") as f:
            print(f"        Downloading [{current_filename}]")
            response = requests.get(file_download_url, stream=True)
            total_length = response.headers.get('content-length')

            if total_length is None: # no content length header
                f.write(response.content)
            else:
                current = 0
                total_length = int(total_length)
                for data in response.iter_content(chunk_size=4096):
                    current += len(data)
                    f.write(data)
                    done = int(50 * current / total_length)
                    downloadPercentage = int(current / total_length * 100)
                    downloadedSize = round((current / 1048576), 2)
                    timePassed = time.perf_counter() - start
                    downloadSpeed = round((downloadedSize / timePassed), 2)
                    totalSize = round((total_length / 1048576), 2)

                    sys.stdout.write(f"\r   [{'#' * done}{' ' * (50-done)}] {downloadPercentage}%  |  Downloaded: [{downloadedSize} MB / {totalSize} MB]  |  Speed: {downloadSpeed} MB/s")
                    sys.stdout.flush()

        print(f"\n        file #{links.index(file_download_url) + 1} download complete.")
    
    except:
        global download_ok
        download_ok = False
        print("\nSomething went wrong!\n")


# start downloads function
def start_download(confirm_start):
    if confirm_start == 'y':
        print("\nStarting download process...")
        for link in links:
            if link != 'exit':
                print(f"file #{links.index(link) + 1}:")
                get_downloads(link)
            else:
                global download_ok
                if download_ok:
                    print("\nEverything downloaded successfully!")
    else:
        print("\nOK! Run the program again if something else to download.\n")


# main function
def main():
    print("\n *** D2GD ***\n")
    print("Enter 'exit' in nothing else to download.\n")

    download_url = input("file #1: Enter download url: ")

    links.append(download_url)

    while download_url != 'exit':
        download_url = input(f"file #{len(links) + 1}: Enter download url: ")
        links.append(download_url)

    confirm_start = input("\nStart downloads? Enter 'y' to agree: ")

    # start process
    start_download(confirm_start);


if __name__ == '__main__':
    main()



---



###Fix file extensions
Files downloaded from indirect URL sometimes doesn't get saved with their appropriate extensions.
<br>

To fix those files...

*   Run below cells sequentially



In [ ]:
pip install filetype

In [ ]:
import os
import filetype
import pathlib

current_path = pathlib.Path().absolute()
print(current_path)

file_list = os.listdir()
print(file_list)


def get_missing_extension():
    for file in file_list:
        if not os.path.splitext(file)[1]:
            print(file)
            filename = file
            kind = filetype.guess(f"{current_path}/{filename}")
            if kind is None:
                print('Cannot guess file type!')
                return
            # print(f"{current_path}/{filename}")
            print(f"\nExtension of {file}: {kind.extension}")
            print(f"MIME type of {file}: {kind.mime}\n")

            file_extension = '.' + kind.extension
            new_filename = filename + file_extension
            os.rename(filename, new_filename)


def main():
    get_missing_extension()
    new_file_list = os.listdir()
    print(new_file_list)


if __name__ == '__main__':
    main()



---



#That's it!